In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
from datetime import datetime
from zoneinfo import ZoneInfo

# -----------------------------
# LOAD DATA
# -----------------------------
df = pd.read_csv("Play Store Data.csv")

# -----------------------------
# CLEAN DATA
# -----------------------------

# Installs → numeric
df["Installs"] = df["Installs"].astype(str).str.replace(r"[+,]", "", regex=True)
df["Installs"] = pd.to_numeric(df["Installs"], errors="coerce")

# Reviews → numeric
df["Reviews"] = pd.to_numeric(df["Reviews"], errors="coerce")

# Size → MB
def size_to_mb(x):
    if isinstance(x, str):
        if "M" in x:
            return float(x.replace("M",""))
        if "k" in x:
            return float(x.replace("k","")) / 1024
    return np.nan

df["Size_MB"] = df["Size"].apply(size_to_mb)

# -----------------------------
# CATEGORY TRANSLATIONS
# -----------------------------
translate_map = {
    "Beauty": "सौंदर्य",        # Hindi
    "Business": "வணிகம்",       # Tamil
    "Dating": "Dating (DE)"
}

df["Category_display"] = df["Category"].replace(translate_map)

# -----------------------------
# FILTER CONDITIONS
# -----------------------------
allowed_categories = [
    "GAME","BEAUTY","BUSINESS","COMICS",
    "COMMUNICATION","DATING","ENTERTAINMENT",
    "SOCIAL","EVENTS"
]

filtered = df[
    (df["Rating"] > 3.5) &
    (df["Reviews"] > 500) &
    (df["Installs"] > 50000) &
    (df["Category"].isin(allowed_categories)) &
    (~df["App"].str.contains("S", case=False, na=False))
]

# sentiment subjectivity column may not exist — safe check
if "Sentiment_Subjectivity" in df.columns:
    filtered = filtered[filtered["Sentiment_Subjectivity"] > 0.5]

# -----------------------------
# TIME CHECK — 5PM to 7PM IST
# -----------------------------
now_ist = datetime.now(ZoneInfo("Asia/Kolkata"))

if 17 <= now_ist.hour < 19:

    fig = px.scatter(
        filtered,
        x="Size_MB",
        y="Rating",
        size="Installs",
        color="Category_display",
        hover_name="App",
        title="Bubble Chart — Size vs Rating vs Installs",
        size_max=60
    )

    # highlight Game category in pink
    for trace in fig.data:
        if "GAME" in trace.name.upper():
            trace.marker.color = "pink"

    fig.show()

else:
    print("Graph hidden — visible only between 5 PM and 7 PM IST")


Graph hidden — visible only between 5 PM and 7 PM IST
